In [ ]:
import requests
import pandas as pd
from datetime import datetime

Api_url_vacancies = "https://api.superjob.ru/2.0/vacancies/"
Api_url_country = "https://api.superjob.ru/2.0/countries/"
Api_url_regions = "https://api.superjob.ru/2.0/regions/"
Api_url_towns = "https://api.superjob.ru/2.0/towns/"
Api_url_reference = "https://api.superjob.ru/2.0/references/"

Api_key = "v3.r.139424290.9dc5b8acc00137024a8b2bf690694888234d63c8.6b844aaeafebd34518dc9a6abb8db56b6307f7d0"

headers = { "X-Api-App-Id": Api_key }

def get_items(url, text):
    params = {"all": True}

    print(f"загрузка {text}...")

    resp = requests.get(url, headers = headers, params = params)

    if resp.status_code == 200:

        data = resp.json()
        objects = data.get('objects', {})
        print(f"загрузили {len(objects)} {text}")
        return objects

    else:
        return None


def get_vacancy(page):

    params = { "page": page }

    response = requests.get(Api_url_vacancies, headers = headers, params = params)

    if response.status_code == 200:
        for vacancy in response.json().get('objects', []):
            date_raw = vacancy.get('date_published', '')
            date_str = (datetime.fromtimestamp(date_raw).strftime('%Y-%m-%d %H:%M')
                if date_raw
                else '')

            town_id = vacancy.get('town', {}).get("id")
            region, country = get_region_id(town_id)
            categ = ""

            if len(vacancy['catalogues']):
                categ = vacancy['catalogues'][0].get("title")

            vacancy_list.append({'id': vacancy['id'],'дата_размещения': date_str,'категория': categ,'название вакансии': vacancy['profession'],
                'зарплата_от': vacancy.get('payment_from', ''),'зарплата_до': vacancy.get('payment_to', ''),'адрес': vacancy.get('address', ''),
                'город': vacancy.get('town', {}).get("title"),'регион': region,'страна': country,'опыт': vacancy.get('experience', {}).get("title"),
                'место работы': vacancy.get('place_of_work', {}).get("title"),'график работы': vacancy.get('type_of_work', {}).get("title"),
                'образование': vacancy.get('education', {}).get("title"),'кандидат': vacancy.get('candidat', '') })

        print(f"загружено {len(vacancy_list)} вакансий")

def get_references():
    print(f"загрузка справочника место работы...")
    resp = requests.get(Api_url_reference, headers = headers)

    if resp.status_code == 200:
        return resp.json()

    else:
        return None

def get_region_name(region_id):

    for region in regions:
        if region['id'] == region_id:
            for country in countries:
                if country['id'] == region['id_country']:

                    return region['title'], country['title']

            return region['title'], None

    else:

        print(f"Ошибка: регион не найден")
    return None

def get_region_id(town_id):

    for town in towns:

        if town['id'] == town_id:
                return get_region_name(town['id_region'])

    else:
        print(f"Ошибка: город не найден")
    return None

countries = get_items(Api_url_country, "стран")
regions = get_items(Api_url_regions, "регионов")
towns = get_items(Api_url_towns, "городов")
references = get_references()

vacancy_list = []

for page in range(25):
    get_vacancy(page)

vacancy_list_sorted = sorted(vacancy_list, key=lambda x: x['дата_размещения'])

df = pd.DataFrame(vacancy_list_sorted)
df.to_csv("vacancies_api.csv", index=False)

загрузка стран...
загрузили 18 стран
загрузка регионов...
загрузили 104 регионов
загрузка городов...
загрузили 5048 городов
загрузка справочника место работы...
загружено 40 вакансий
загружено 80 вакансий
загружено 120 вакансий
загружено 160 вакансий
загружено 200 вакансий
загружено 240 вакансий
загружено 280 вакансий
загружено 320 вакансий
загружено 360 вакансий
загружено 400 вакансий
загружено 440 вакансий
загружено 480 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий
загружено 520 вакансий


In [ ]:
df.shape

(520, 15)

In [ ]:
from google.colab import files
files.download("vacancies_api.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>